In [1]:
import tensorflow as tf

import os
import pathlib
import time
import datetime

import numpy as np

from matplotlib import pyplot as plt
from IPython import display

import numpy as np
from sklearn.model_selection import GridSearchCV

import sys

sys.path.append('/home/natanael/deblurring_images')
from data import Dataset
from models.pixpix import Pix2Pix
from loss_function import Loss
from saved_results import SavedResults
from optimizers import Optimizers
from gridsearch import GridSearch

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'


2023-07-22 12:44:14.281785: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# Get the list of physical GPU devices
physical_devices = tf.config.list_physical_devices('GPU')

# Set memory growth for each physical GPU (if available)
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

2023-07-22 12:44:16.364107: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-07-22 12:44:16.364883: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-07-22 12:44:16.380307: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-22 12:44:16.380475: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.2GHz coreCount: 30 deviceMemorySize: 5.78GiB deviceMemoryBandwidth: 245.91GiB/s
2023-07-22 12:44:16.380495: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-07-22 12:44:16.382066: I tensorflow/stream_executor/platform/defa

In [3]:
dataset = Dataset(buffer_size=400, batch_size=1 )

networks = [Pix2Pix]
loss_discriminator = ['disc_loss',]
loss_generator = ['pix2pix_loss',]
opt_gen_names = ['Adam','AdamX', 'RMSprop', 'SGD']
lrs_gen = [0.0002,]
bts_1_gen = [0.2,]
bts_2_gen = [0.9999,]

opts_disc_name =['Adam','AdamX', 'RMSprop', 'SGD']
lrs_disc = [0.0002,]
bts_1_disc = [0.2,]
bts_2_disc = [0.9999,]

2023-07-22 12:44:16.399992: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-22 12:44:16.400651: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-07-22 12:44:16.400884: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-22 12:44:16.401113: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.2GHz coreCount: 30 deviceMemorySize: 5.78GiB deviceMemoryBandwidth: 245.91GiB

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


efault/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2023-07-22 12:44:16.401270: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcufft.so.10
2023-07-22 12:44:16.401297: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcurand.so.10
2023-07-22 12:44:16.401325: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcusolver.so.10
2023-07-22 12:44:16.401353: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcusparse.so.10
2023-07-22 12:44:16.401382: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2023-07-22 12:44:16.401510: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so retur

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
base loaded.


In [4]:
grid = GridSearch(
    networks=networks, 
    dataset=dataset,
    steps=1000,
    loss_discriminator=loss_discriminator,
    loss_generator=loss_generator,
    opt_gen_names=opt_gen_names,
    lrs_gen=lrs_gen,
    bts_1_gen=bts_1_gen,
    bts_2_gen=bts_2_gen,
    opts_disc_name=opts_disc_name,
    lrs_disc=lrs_disc,
    bts_1_disc=bts_1_disc,
    bts_2_disc=bts_2_disc )

grid.run()

Step: 0k
....................................................................................................

In [5]:
SavedResults().show_dataframe_sorted_by_mse()

,id,mse,ssim,parameters
12,1,0.025202,0.354245,"Loss: pix2pix_loss, Optimazer:{'name': 'SGD', ..."
6,1,0.020889,0.472450,"Loss: pix2pix_loss, Optimazer:{'name': 'Adamax..."
13,1,0.020772,0.358211,"Loss: pix2pix_loss, Optimazer:{'name': 'SGD', ..."
5,1,0.019661,0.492270,"Loss: pix2pix_loss, Optimazer:{'name': 'Adamax..."
4,1,0.019612,0.506098,"Loss: pix2pix_loss, Optimazer:{'name': 'Adamax..."
7,1,0.019602,0.502072,"Loss: pix2pix_loss, Optimazer:{'name': 'Adamax..."
14,1,0.019476,0.361661,"Loss: pix2pix_loss, Optimazer:{'name': 'SGD', ..."
15,1,0.018212,0.406186,"Loss: pix2pix_loss, Optimazer:{'name': 'SGD', ..."
1,1,0.016000,0.558271,"Loss: pix2pix_loss, Optimazer:{'name': 'Adam',..."
8,1,0.015289,0.549159,"Loss: pix2pix_loss, Optimazer:{'name': 'RMSpro..."
